<a href="https://colab.research.google.com/github/nfpaiva/ml-ai-experiments/blob/main/notebooks/netron-dnn-visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q onnx

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
import torch
import torch.nn as nn
# from torchviz import make_dot
from IPython.display import display, Image

# Define model for auto insurance unique features
class AutoInsuranceModel(nn.Module):
    def __init__(self):
        super(AutoInsuranceModel, self).__init__()
        self.fc1 = nn.Linear(1, 16)  # Assuming 1 unique feature for auto
        self.fc2 = nn.Linear(16, 8)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        return x

# Define model for home insurance unique features
class HomeInsuranceModel(nn.Module):
    def __init__(self):
        super(HomeInsuranceModel, self).__init__()
        self.fc1 = nn.Linear(1, 16)  # Assuming 1 unique feature for home
        self.fc2 = nn.Linear(16, 8)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        return x

# Define model for common features
class CommonFeaturesModel(nn.Module):
    def __init__(self):
        super(CommonFeaturesModel, self).__init__()
        self.fc1 = nn.Linear(2, 16)  # Assuming 2 common features
        self.fc2 = nn.Linear(16, 8)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        return x

# Define the combined model
class CombinedInsuranceModel(nn.Module):
    def __init__(self):
        super(CombinedInsuranceModel, self).__init__()
        self.auto_model = AutoInsuranceModel()
        self.home_model = HomeInsuranceModel()
        self.common_model = CommonFeaturesModel()
        self.fc1 = nn.Linear(24, 8)  # Adjusting input size to match concatenated features
        self.fc2 = nn.Linear(8, 1)

    def forward(self, common_input, auto_input=None, home_input=None):
        common_features = self.common_model(common_input)

        if auto_input is not None:
            auto_features = self.auto_model(auto_input)
        else:
            auto_features = torch.zeros_like(common_features)

        if home_input is not None:
            home_features = self.home_model(home_input)
        else:
            home_features = torch.zeros_like(common_features)

        # Concatenate features from all paths
        combined_features = torch.cat((common_features, auto_features, home_features), dim=1)

        # Apply linear layers
        combined_output = nn.functional.relu(self.fc1(combined_features))
        output = torch.sigmoid(self.fc2(combined_output))
        return output


In [ ]:
# export model to viz at https://netron.app/
# Initialize the model
model = CombinedInsuranceModel()

# Dummy input for exporting the model
common_input = torch.randn(1, 2)  # Batch size of 1 and 2 common features
auto_input = torch.randn(1, 1)    # Batch size of 1 and 1 auto feature
home_input = torch.randn(1, 1)    # Batch size of 1 and 1 home feature

# Export the model to an ONNX file
torch.onnx.export(model,
                  (common_input, auto_input, home_input),
                  "combined_insurance_model.onnx",
                  input_names=['common_input', 'auto_input', 'home_input'],
                  output_names=['output'],
                  dynamic_axes={'common_input': {0: 'batch_size'},
                                'auto_input': {0: 'batch_size'},
                                'home_input': {0: 'batch_size'},
                                'output': {0: 'batch_size'}})

print("Model exported to combined_insurance_model.onnx")

Model exported to combined_insurance_model.onnx


In [ ]:
# Define the model
model = CombinedInsuranceModel()

# Example data for training
common_inputs = torch.tensor([[3, 40], [3, 40]], dtype=torch.float32)  # Batch size of 2, common features
auto_inputs = torch.tensor([[15], [0]], dtype=torch.float32)  # Batch size of 2, auto-specific feature (0 for missing)
home_inputs = torch.tensor([[0], [30]], dtype=torch.float32)  # Batch size of 2, home-specific feature (0 for missing)
targets = torch.tensor([[1], [0]], dtype=torch.float32)  # Target values

# Zero out auto/home inputs where not applicable
auto_inputs[1, :] = 0  # Set second row (home insurance) auto input to 0
home_inputs[0, :] = 0  # Set first row (auto insurance) home input to 0

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()

    # Forward pass
    outputs = model(common_inputs, auto_input=auto_inputs, home_input=home_inputs)

    # Compute the loss
    loss = criterion(outputs, targets)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete.")


Epoch [100/1000], Loss: 0.0394
Epoch [200/1000], Loss: 0.0045
Epoch [300/1000], Loss: 0.0017
Epoch [400/1000], Loss: 0.0009
Epoch [500/1000], Loss: 0.0006
Epoch [600/1000], Loss: 0.0004
Epoch [700/1000], Loss: 0.0003
Epoch [800/1000], Loss: 0.0002
Epoch [900/1000], Loss: 0.0002
Epoch [1000/1000], Loss: 0.0001
Training complete.
